In [1]:
import folium
import geopy
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import pandas as pd
from collections import defaultdict

In [2]:
class Point:
    def __init__(self, latitude, longitude):
        self.latitude = latitude
        self.longitude = longitude

In [3]:
points = []
points.clear()
csv = pd.read_csv("Coordinates.csv")
for index, row in csv.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    points.append(Point(latitude, longitude))

In [4]:
def distance(point1, point2):
    return geodesic((point1.latitude, point1.longitude), (point2.latitude, point2.longitude)).kilometers

In [5]:
def totalDistance(order):
    totalDistance = 0
    numPoints = len(order)
    for i in range(numPoints - 1):
        point1 = order[i]
        point2 = order[i + 1]
        distance1 = distance(point1, point2)
        totalDistance += distance1
    return totalDistance

In [6]:
numPoints = len(points)
graph = [[0] * numPoints for _ in range(numPoints)]
for i in range(numPoints):
    for j in range(i + 1, numPoints):
        dist = distance(points[i], points[j])
        graph[i][j] = graph[j][i] = dist
parent = list(range(numPoints))
print(parent)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [7]:
#Find MST via Kruskal's Algorithm
def kruskalMST(graph, points):
    numPoints = len(points)
    edges = []
    for i in range(numPoints):
        for j in range(i + 1, numPoints):
            if graph[i][j] > 0:
                edges.append((i, j, graph[i][j]))
    edges.sort(key = lambda edge: edge[2])
    for i in range(numPoints):
        parent[i] = i
        mst = []
    for edge in edges:
        u, v, weight = edge
        parent_u = u
        parent_v = v
        
        
        while parent[parent_u] != parent_u:
            parent_u = parent[parent_u]
        while parent[parent_v] != parent_v:
            parent_v = parent[parent_v]
        
        if parent_u != parent_v:  # Adding the edge doesn't create a cycle
            mst.append(edge)
            parent[parent_u] = parent_v
        
        if len(mst) == numPoints - 1:
            break
    return mst
mst = kruskalMST(graph, points)
print(mst)
print(len(mst))

[(2, 17, 0.3475406831976341), (3, 17, 0.48497802447383803), (8, 19, 0.7674980426778136), (9, 19, 0.7952759167415934), (4, 5, 0.8740916853693048), (10, 11, 0.8887283074480037), (13, 16, 0.9345080143827359), (15, 16, 0.942748078690881), (0, 1, 1.0345964423925742), (11, 18, 1.0569017169866488), (5, 6, 1.1404688392426023), (1, 2, 1.1961097949432005), (12, 13, 1.2954759296954397), (4, 8, 1.39955442155937), (1, 13, 1.4852327633552842), (3, 4, 1.5091137032579094), (14, 15, 1.896430613769071), (5, 7, 1.9859297704356549), (7, 11, 2.583975842235388)]
19


In [8]:
Map = folium.Map(location=[points[0].latitude, points[0].longitude], zoom_start=12)

In [9]:
for edge in mst:
    u, v, weight = edge
    folium.PolyLine([(points[u].latitude, points[u].longitude), (points[v].latitude, points[v].longitude)],
                    color='blue', weight=2.5, opacity=1).add_to(Map)
count = 0
for point in points:
    folium.Marker(location = [point.latitude, point.longitude], 
                  popup = [point.latitude, point.longitude, count], 
                  icon=folium.Icon(color='blue', icon='map-marker')).add_to(Map)
    count+=1
folium.Marker(location = [points[0].latitude, points[0].longitude], 
                  popup = [points[0].latitude, points[0].longitude, 0], 
                  icon=folium.Icon(color='red', icon='map-marker')).add_to(Map)
Map

In [10]:
def oddDegreeNodes(mst):
    degrees = defaultdict(int)

    for fromNode, toNode, weight in mst:
        degrees[fromNode] += 1
        degrees[toNode] += 1

    oddNodes = set()
    for node, degree in degrees.items():
        if degree % 2 == 1:
            oddNodes.add(node)

    return oddNodes

In [11]:
oddNodes = oddDegreeNodes(mst)
print("Points with Odd Degree:", oddNodes)
print(len(oddNodes))

Points with Odd Degree: {0, 1, 4, 5, 6, 9, 10, 11, 12, 13, 14, 18}
12


In [12]:
Map = folium.Map(location=[points[0].latitude, points[0].longitude], zoom_start=12)
for point in oddNodes:
    folium.Marker(location = [points[point].latitude, points[point].longitude], 
                  popup = [points[point].latitude, points[point].longitude], 
                  icon=folium.Icon(color='blue', icon='map-marker')).add_to(Map)
Map

In [13]:
#Perfect match by linking the shortest distances between the odd nodes; suboptimal but works
#stuck here for a while due to lack of intuitive algorithm's for perfect matching
def greedyPerfectMatching(oddNodes, points):
    edges = []
    
    for i in oddNodes:
        for j in oddNodes:
            if i < j:
                dist = distance(points[i], points[j])
                edges.append((i, j, dist))
    
    # Sort edges by distance
    edges.sort(key=lambda x: x[2])
    
    matched = []
    matching = []
    
    for edge in edges:
        u, v, dist = edge
        if u not in matched and v not in matched:
            matching.append(edge)
            matched.append(u)
            matched.append(v)
    
    return matching
matched = greedyPerfectMatching(oddNodes, points)
print(matched)


Map = folium.Map(location=[points[0].latitude, points[0].longitude], zoom_start=12)

for edge in matched:
    u, v, weight = edge
    folium.PolyLine([(points[u].latitude, points[u].longitude), (points[v].latitude, points[v].longitude)],
                    color='blue', weight=2.5, opacity=1).add_to(Map)
for i in oddNodes:
    folium.Marker(location=[points[i].latitude, points[i].longitude],
                  popup=[points[i].latitude, points[i].longitude],
                  icon=folium.Icon(color='blue', icon='map-marker')).add_to(Map)
Map

[(4, 5, 0.8740916853693048), (10, 11, 0.8887283074480037), (0, 1, 1.0345964423925742), (12, 13, 1.2954759296954397), (6, 18, 3.6501034523559066), (9, 14, 6.7704933045381095)]


In [14]:
#Create eulerian tour with Hierholzer's Algorithm
def createEulerianTour(multigraph, numPoints):
    graph = defaultdict(list)
    for u, v, weight in multigraph:
        graph[u].append(v)
        graph[v].append(u)
    
    #startVertex = 0
    #for u in range(numPoints):
    #    if graph[u]:
    #        startVertex = u
    #        break
    
    #if startVertex is None:
    #    return []
    
    currPath = [0]
    path = []
    
    while currPath:
        u = currPath[-1]
        if graph[u]:
            v = graph[u].pop()
            graph[v].remove(u)
            currPath.append(v)
        else:
            path.append(currPath.pop())
        
    return path[::-1]

# Create the multigraph with MST and perfect matching
multigraph = mst.copy()
for u, v, weight in matched:
    multigraph.append((u, v, weight))

eulerianTour = createEulerianTour(multigraph, numPoints)
print("Eulerian Tour:", eulerianTour)
print(len(eulerianTour))

Map = folium.Map(location=[points[0].latitude, points[0].longitude], zoom_start=12)

for i in range(len(eulerianTour) - 1):
    u = eulerianTour[i]
    v = eulerianTour[i + 1]
    folium.PolyLine([(points[u].latitude, points[u].longitude), (points[v].latitude, points[v].longitude)],
                    color='blue', weight=2.5, opacity=1).add_to(Map)
count = 0
for point in points:
    folium.Marker(location=[point.latitude, point.longitude],
                  popup=[point.latitude, point.longitude, count],
                  icon=folium.Icon(color='blue', icon='map-marker')).add_to(Map)
    count+=1

folium.Marker(location=[points[0].latitude, points[0].longitude],
              popup=[points[0].latitude, points[0].longitude],
              icon=folium.Icon(color='red', icon='map-marker')).add_to(Map)

Map

Eulerian Tour: [0, 1, 13, 12, 13, 16, 15, 14, 9, 19, 8, 4, 5, 7, 11, 10, 11, 18, 6, 5, 4, 3, 17, 2, 1, 0]
26


In [15]:
# Shortcut the eulerian circuit and creating a hamiltonion circuit
def createHamiltonionCircuit(eulerianTour):
    visited = []
    hamiltonionCircuit = []
    for v in eulerianTour:
        if v not in visited:
            visited.append(v)
            hamiltonionCircuit.append(v)
    hamiltonionCircuit.append(hamiltonionCircuit[0]) #don't forget last point
    return hamiltonionCircuit

hamiltonionCircuit = createHamiltonionCircuit(eulerianTour)
print("Hamiltonion Circuit: ", hamiltonionCircuit)


Map = folium.Map(location=[points[0].latitude, points[0].longitude], zoom_start=12)

for i in range(len(hamiltonionCircuit) - 1):
    u = hamiltonionCircuit[i]
    v = hamiltonionCircuit[i + 1]
    folium.PolyLine([(points[u].latitude, points[u].longitude), (points[v].latitude, points[v].longitude)],
                    color='blue', weight=2.5, opacity=1).add_to(Map)
count = 0
for point in points:
    folium.Marker(location=[point.latitude, point.longitude],
                  popup=[point.latitude, point.longitude, count],
                  icon=folium.Icon(color='blue', icon='map-marker')).add_to(Map)
    count+=1

folium.Marker(location=[points[0].latitude, points[0].longitude],
              popup=[points[0].latitude, points[0].longitude],
              icon=folium.Icon(color='red', icon='map-marker')).add_to(Map)

Map

Hamiltonion Circuit:  [0, 1, 13, 12, 16, 15, 14, 9, 19, 8, 4, 5, 7, 11, 10, 18, 6, 3, 17, 2, 0]


In [16]:
def toPoints(hamiltonionCircuit):
    christofidesPath = []
    for i in range(len(hamiltonionCircuit) - 2):
        christofidesPath.append(points[i])
    return christofidesPath
christofidesPath = toPoints(hamiltonionCircuit)
christofidesPath.append(points[0])
print(christofidesPath)
print(len(christofidesPath))
print(totalDistance(christofidesPath))

[<__main__.Point object at 0x108ce6990>, <__main__.Point object at 0x105bf8e60>, <__main__.Point object at 0x1088299a0>, <__main__.Point object at 0x105ba6bd0>, <__main__.Point object at 0x107653a70>, <__main__.Point object at 0x107d11fa0>, <__main__.Point object at 0x108e45fd0>, <__main__.Point object at 0x108e45310>, <__main__.Point object at 0x10837d370>, <__main__.Point object at 0x1068c6f60>, <__main__.Point object at 0x105c1cf80>, <__main__.Point object at 0x107b52120>, <__main__.Point object at 0x1065e6030>, <__main__.Point object at 0x105bd5040>, <__main__.Point object at 0x108dbe900>, <__main__.Point object at 0x108e55460>, <__main__.Point object at 0x108e55550>, <__main__.Point object at 0x108e545f0>, <__main__.Point object at 0x108e55490>, <__main__.Point object at 0x108ce6990>]
20
54.668974536739455


In [17]:
def twoOpt(points, maxIterationsWithoutImprovement):
    iterationsWithoutImprovement = 0
    while iterationsWithoutImprovement < maxIterationsWithoutImprovement:
        improved = False
        for i in range(1, len(points) - 1):
            for j in range(i + 1, len(points)):
                newPath = points.copy()
                
                #print('old path------------')
                #for p in newPath:
                #    print(p.index)

                newPath[i:j] = points[j-1:i-1:-1]
                #swaps the points from i inclusive to j exclusive i.e 1,2,3,4,5 where i = 1 j = 4 becomes 1,4,3,2,5

                
                #print('new ------------')
                #for p in newPath:
                #    print(p.index)
                
                oldDistance = distance(points[i - 1], points[i]) + distance(points[j - 1], points[j])
                newDistance = distance(points[i - 1], points[j - 1]) + distance(points[i], points[j])
                #a = oldDistance - newDistance

                #oldTotalDis = totalDistance(points)
                #newTotalDis = totalDistance(newPath)
                #b = oldTotalDis - newTotalDis
                
                #if abs(a - b) > 0.000001:
                #    print ("error " + str(a - b))
                
                #print(oldDistance)
                #print(newDistance)
                if newDistance < oldDistance:
                    points = newPath
                    improved = True
                    iterationsWithoutImprovement = 0
                if not improved:
                    iterationsWithoutImprovement += 1
    return points
christofidesTwoOpt = twoOpt(christofidesPath, 10)

In [18]:
Map = folium.Map(location=[points[0].latitude, points[0].longitude], zoom_start=12)
for i in range(len(christofidesTwoOpt) - 1):
    u = christofidesTwoOpt[i]
    v = christofidesTwoOpt[i + 1]
    folium.PolyLine([(u.latitude, u.longitude), (v.latitude, v.longitude)],
                    color='blue', weight=2.5, opacity=1).add_to(Map)

for point in points:
    folium.Marker(location=[point.latitude, point.longitude],
                  popup=[point.latitude, point.longitude],
                  icon=folium.Icon(color='blue', icon='map-marker')).add_to(Map)

folium.Marker(location=[points[0].latitude, points[0].longitude],
              popup=[points[0].latitude, points[0].longitude],
              icon=folium.Icon(color='red', icon='map-marker')).add_to(Map)

Map

In [19]:
print(totalDistance(christofidesTwoOpt))

30.520279510255858
